In [ ]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from xgboost import XGBRegressor
import pandas as pd
import numpy as np
import re
import pickle
import matplotlib.pyplot as plt
from xgboost import plot_importance
import plotly.graph_objects as go
from tqdm.notebook import tqdm
from sklearn.model_selection import RepeatedKFold
import xgboost as xgb
from sklearn.metrics import roc_auc_score, roc_curve, f1_score, accuracy_score, recall_score, precision_score, confusion_matrix, mean_squared_error
import seaborn as sns
from statannot import add_stat_annotation
from sklearn.linear_model import Lasso, MultiTaskLasso, Ridge, ElasticNet

from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.feature_selection import mutual_info_regression, f_regression

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedKFold
from sklearn.decomposition import PCA

from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler

import time
import scipy
from sklearn import metrics

from sklearn.svm import SVR

from skopt import BayesSearchCV

from statsmodels.stats.multitest import fdrcorrection

from statsmodels.stats import multitest

## Biblioteca

In [ ]:



def get_color(var, X_cat_, color_dict_):

    X_factors = list(X_cat_.factors)
    X_name = list(X_cat_.newname)

    index_code = X_name.index(var)
    cat = X_factors[index_code]

    return color_dict_[cat]

color_dict = {}
color_dict['Country'] = '#484849'
color_dict['HF'] = '#A31300'
color_dict['LSF'] = '#FF9E0D'
color_dict['PSF'] = '#FF4800'
color_dict['SDHF'] = '#1C4F9E'
color_dict['DF'] = '#009E32'

def get_bar_colors(data_, X_cat_):
    color_dict = {}
    color_dict['FP'] = '#111111'
    color_dict['HF'] = '#A31300'
    color_dict['FF1'] = '#FF9E0D'
    color_dict['CF'] = '#FF4800'
    color_dict['FF'] = '#1C4F9E'
    color_dict['DF'] = '#009E32'
    color_dict['FlairFL'] = '#DAEB07'
    color_dict['FlairFR'] = '#EB9700'
    color_dict['FlairF'] = '#FFD400'
    color_dict['GMF'] = '#565E75'
    
    
    #color_dict = {}
    #color_dict = {'DF':'#5975a4', 'MF':'#cc8963', 'SF':'#5f9e6e', 'SLF':'#b55d60',
    #              'n':'#857aab', 'country':'#8d7866'}#, '#d095bf'}

    X_factors = list(X_cat_.factors)
    X_name = list(X_cat_.name)

    bar_color_total = []
    for i in list(data_.Features):
        if(i == 'Country'):
            bar_color_total.append(color_dict['Country'])
            continue
        index_code = X_name.index(i)
        cat = X_factors[index_code]
        bar_color_total.append(color_dict[cat])
    return bar_color_total


def adj_r2_score_and_r2_score(clf, X, y):
    n = X.shape[0]  # Number of observations
    p = X.shape[1]  # Number of features
    r_squared = r2_score(y, clf.predict(X))
    return [1 - (1 - r_squared) * ((n - 1) / (n - p - 1)), r_squared]


def mse(clf, X, y):
    return mean_squared_error(y, clf.predict(X))

def rmse(clf, X, y):
    mse = mean_squared_error(y, clf.predict(X))
    return math.sqrt(mse)    

def coef_se(clf, X, y):
    n = X.shape[0]
    X1 = np.hstack((np.ones((n, 1)), np.matrix(X)))
    se_matrix = scipy.linalg.sqrtm(
        metrics.mean_squared_error(y, clf.predict(X)) *
        np.linalg.inv(X1.T * X1)
    )
    return np.diagonal(se_matrix)

def coef_tval(clf, X, y):
    a = np.array(clf.intercept_ / coef_se(clf, X, y)[0])
    b = np.array(clf.coef_ / coef_se(clf, X, y)[1:])
    return np.append(a, b)

def coef_tval_SVR_tree(clf, X, y):
    a = np.array(clf.intercept_ / coef_se(clf, X, y)[0])
    b = np.array(clf._get_coef() / coef_se(clf, X, y)[1:])
    return np.append(a, b)

def coef_pval(clf, X, y):

    n = X.shape[0]
    t = coef_tval(clf, X, y)
    p = 2 * (1 - scipy.stats.t.cdf(abs(t), n - 1))
    return p

def coef_pval_SVR_tree(clf, X, y):

    n = X.shape[0]
    t = coef_tval_SVR_tree(clf, X, y)
    p = 2 * (1 - scipy.stats.t.cdf(abs(t), n - 1))
    return p

def residuals(clf, X, y, r_type='standardized'):

    # Make sure value of parameter 'r_type' is one we recognize
    assert r_type in ('raw', 'standardized', 'studentized'), (
        "Invalid option for 'r_type': {0}".format(r_type))
    y_true = y.view(dtype='float')
    # Use classifier to make predictions
    y_pred = clf.predict(X)
    # Make sure dimensions agree (Numpy still allows subtraction if they don't)
    assert y_true.shape == y_pred.shape, (
        "Dimensions of y_true {0} do not match y_pred {1}".format(y_true.shape,
                                                                  y_pred.shape))
    # Get raw residuals, or standardized or standardized residuals
    resids = y_pred - y_true
    if r_type == 'standardized':
        resids = resids / np.std(resids)
    elif r_type == 'studentized':
        # Prepare a blank array to hold studentized residuals
        studentized_resids = np.zeros(y_true.shape[0], dtype='float')
        # Calcluate hat matrix of X values so you can get leverage scores
        hat_matrix = np.dot(
            np.dot(X, np.linalg.inv(np.dot(np.transpose(X), X))),
            np.transpose(X))
        # For each point, calculate studentized residuals w/ leave-one-out MSE
        for i in range(y_true.shape[0]):
            # Make a mask so you can calculate leave-one-out MSE
            mask = np.ones(y_true.shape[0], dtype='bool')
            mask[i] = 0
            loo_mse = np.average(resids[mask] ** 2, axis=0)  # Leave-one-out MSE
            # Calculate studentized residuals
            studentized_resids[i] = resids[i] / np.sqrt(
                loo_mse * (1 - hat_matrix[i, i]))
        resids = studentized_resids
    return resids


def f_squared(clf, X, y):

    n = X.shape[0]
    p = X.shape[1]
    r_squared = metrics.r2_score(y, clf.predict(X))
    return r_squared  / (1 - r_squared)

def f_stat(clf, X, y):
    """Calculate summary F-statistic for beta coefficients.

    Parameters
    ----------
    clf : sklearn.linear_model
        A scikit-learn linear model classifier with a `predict()` method.
    X : numpy.ndarray
        Training data used to fit the classifier.
    y : numpy.ndarray
        Target training values, of shape = [n_samples].

    Returns
    -------
    float
        The F-statistic value.
    """
    n = X.shape[0]
    p = X.shape[1]
    r_squared = metrics.r2_score(y, clf.predict(X))
    return (r_squared / p) / ((1 - r_squared) / (n - p - 1))


def f_stat_pvalue(clf, X, y):
    """Calculate summary F-statistic p value for beta coefficients.

    Parameters
    ----------
    clf : sklearn.linear_model
        A scikit-learn linear model classifier with a `predict()` method.
    X : numpy.ndarray
        Training data used to fit the classifier.
    y : numpy.ndarray
        Target training values, of shape = [n_samples].

    Returns
    -------
    float
        The F-statistic p value.
    """
    n = X.shape[0] # Esto se extrae par los grados de libertad del numeador y el denomindor (no. predictores, no. sujetos - no. predictores-1)
    p = X.shape[1]
    r_squared = metrics.r2_score(y, clf.predict(X))
    
    return np.round(scipy.stats.f.sf(f_stat(clf, X, y), n, (n - p - 1)), 15)

def compute_f_statistics(clf, X, y):
    return [f_stat(clf, X, y), f_stat_pvalue(clf, X, y)]



def boostrapping(X_func, case1A_func, case1A_func_for_strat, lista_best_func, model_ = SVR(kernel='rbf', degree=4), iter_total = 1000):
    print(model_)
    
    coef_array_func = np.zeros([len(lista_best_func)+1, iter_total])
    coef_t_value_func = np.zeros([len(lista_best_func)+1, iter_total])
    coef_p_value_func = np.zeros([len(lista_best_func)+1, iter_total])

    r2_list_func =[]
    f2_list_func =[]
    f_list_func =[]
    f_p_value_list_func =[]
    for iter in tqdm(range(0,iter_total), total=len(range(0, iter_total)),leave=True, mininterval=1, ascii=True,
                            desc='Training model'):
        # Stratified split
        
        finish = True
        
        while(finish):
            
            seed = np.random.randint(100, 200)

            X_train, X_test, y_train, y_test = train_test_split(X_func, case1A_func, stratify=case1A_func_for_strat,test_size=0.30, random_state=seed)

            model_func = model_

            model_func.fit(X_train, y_train)

            coef_array_func[0,iter] = np.abs(model_func.intercept_)
            coef_array_func[1::,iter] = np.abs(model_func._get_coef())

            try:

                coef_t_value_func[:,iter] = np.round(np.abs(coef_tval_SVR_tree(model_func, X_test, y_test)), 30)
                coef_p_value_func[:,iter] = np.round(np.abs(coef_pval_SVR_tree(model_func, X_test, y_test)), 30)


                f2_list_func.append(f_squared(model_func, X_test, y_test))
                f_list_func.append(f_stat(model_func, X_test, y_test))
                f_p_value_list_func.append(f_stat_pvalue(model_func, X_test, y_test))
                r2_list_func.append(r2_score(y_test, model_func.predict(X_test)))
                finish = False
            except:
                finish = True


    r2_list_array = np.array(r2_list_func)
    # finding the 1st quartile
    q1 = np.quantile(r2_list_array, 0.25)
 
    # finding the 3rd quartile
    q3 = np.quantile(r2_list_array, 0.75)
    med = np.median(r2_list_array)

    # finding the iqr region
    iqr = q3-q1

    # finding upper and lower whiskers
    upper_bound = q3+(1.5*iqr)
    lower_bound = q1-(1.5*iqr)
    #print(iqr, upper_bound, lower_bound)

    index_del = []     

    for i in range(r2_list_array.shape[0]):
        if((r2_list_array[i]<= lower_bound) | (r2_list_array[i] >= upper_bound)):
            index_del.append(i)

    
    
    r2_list_func = list(np.delete(r2_list_array,index_del, axis=0 ))

    coef_df_func = pd.DataFrame(
            index=['_intercept'] + lista_best_func,
            columns=['Estimate','t value', 'p value', 'p value mean']
        )

    coef_array_func  = np.delete(coef_array_func,index_del, axis=1 )
    coef_t_value_func = np.delete(coef_t_value_func,index_del, axis=1 )
    coef_p_value_func = np.delete(coef_p_value_func,index_del, axis=1 )
    
    
    coef_array_func_mean = np.zeros([len(lista_best_func)+1, 1])
    coef_t_value_func_mean = np.zeros([len(lista_best_func)+1, 1])
    coef_p_value_func_mean = np.zeros([len(lista_best_func)+1, 1])
    coef_p_value_func_fdr = np.zeros([len(lista_best_func)+1, 1])
    
    coef_array_func_std = np.zeros([len(lista_best_func)+1, 1])

    print('shape',coef_array_func.shape, coef_t_value_func.shape, coef_p_value_func.shape, len(index_del))
    
    for i in range(len(lista_best_func)+1):
        coef_array_func_mean[i] = coef_array_func[i,:].mean()
        coef_t_value_func_mean[i]= coef_t_value_func[i,:].mean()
        coef_p_value_func_mean[i]= coef_p_value_func[i,:].mean()
        coef_p_value_func_fdr[i]= fdrcorrection(coef_p_value_func[i,:], alpha=0.05, is_sorted=False )[1].max()
        
        coef_array_func_std[i] = coef_array_func[i,:].std()
        # Se corrige por FDR y se reporta el valor máximo

    coef_df_func['Estimate'] = coef_array_func_mean
    coef_df_func['t value'] = coef_t_value_func_mean
    coef_df_func['p value'] = coef_p_value_func_fdr
    coef_df_func['p value mean'] = coef_p_value_func_mean
    coef_df_func['Estimate std'] = coef_array_func_std


    empty_str = []
    for i in range(coef_df_func.shape[0]):
        empty_str.append('')

    coef_df_func['value'] = empty_str

    coef_df_func = coef_df_func.T
    coef_df_func['R-squared'] = ['','','','', '', np.mean(r2_list_func)]
    coef_df_func['CI'] = ['','','','', '', np.std(r2_list_func)*1.96]
    coef_df_func['F-squared'] = ['','','','', '', np.mean(f2_list_func)]
    coef_df_func['F'] = ['','','','', '', np.mean(f_list_func)]
    
    
    pvalue_limit = 1e-300

    # Ajusta valores p que son menores que el límite
    adjusted_pvalues = [p if p > pvalue_limit else pvalue_limit for p in fdrcorrection(f_p_value_list_func, alpha=0.05, method='indep', is_sorted=False )[1]]
    from scipy.stats import combine_pvalues
    
    
    coef_df_func['F-pvalue'] = ['','','','', '', combine_pvalues(adjusted_pvalues, method='fisher')[1]]
    
    #coef_df_func['F-pvalue_fdr'] = ['','','', fdrcorrection(f_p_value_list_func, alpha=0.05, method='indep', is_sorted=False )]
    coef_df_func = coef_df_func.T


    print(np.mean(r2_list_func),  np.std(r2_list_func)*1.96)
    
    return [coef_df_func, r2_list_func, f_p_value_list_func, fdrcorrection(f_p_value_list_func, alpha=0.05, is_sorted=False )]


def plot_estimate_value(regression_model, X_cat_ = [], title = '',  xlim =[0, 2] ,fig_size = (8,12), size = 12, pvalue_type = 'False', plot_ci = ''):

    df = regression_model
    df.index.name = 'Features'
    df = df.iloc[1:-5, 0:-1]

    for i in range(3):
            #print(i, X_RAW_edu_level[X_RAW_edu_level.columns[i]].dtype)
        df[df.columns[i]] = np.abs(pd.to_numeric(df[df.columns[i]],errors = 'coerce'))

    df = df.reset_index()
    data = df.sort_values('Estimate', ascending=False)

    
    if(len(X_cat_)>0):
        
        bar_color = get_bar_colors(data, X_cat_)
        
        plt.title(title)
        sns.barplot(x="Estimate", y="Features", data = data, xerr=data['Estimate std'], palette =bar_color)
        plt.xlim(xlim)
    else:
        plt.title(title)
        sns.barplot(x="Estimate", y="Features", data = data, color = 'darkblue')
        plt.xlim(xlim)


    y_step=0  
    for i in range(df.shape[0]):
        if(np.round(df.sort_values('Estimate', ascending=False)['p value'].iloc[y_step],3)<=0.01):
            color = 'green'
        elif(np.round(df.sort_values('Estimate', ascending=False)['p value'].iloc[y_step],3)<=0.05):
            color = 'gray'
        else:
            color = 'red'        
        
        if(pvalue_type == 'color'):
                plt.text(df.sort_values('Estimate', ascending=False)['Estimate'].iloc[y_step]-0.005, y_step, 
                                 '' + str(np.round(df.sort_values('Estimate', ascending=False)['p value'].iloc[y_step],2)),
                                 size= size, rotation=0.,
                                 ha="left", va="center", color = 'white',
                                 bbox=dict(boxstyle="round",
                                           ec=color,
                                            fc=color,
                                           )
                                 )
                y_step+=1

        elif(pvalue_type == 'value'):
                plt.text(df.sort_values('Estimate', ascending=False)['Estimate'].iloc[y_step]+0.005, y_step, 
                                 '(' + str(np.round(df.sort_values('Estimate', ascending=False)['p value'].iloc[y_step],8))+')',
                                 size= size, rotation=0.,
                                 ha="left", va="center", color = 'black',

                                 )
                y_step+=1
        else:
            
            if(df.sort_values('Estimate', ascending=False)['p value'].iloc[y_step]<0.01):
                
                    plt.text(df.sort_values('Estimate', ascending=False)['Estimate'].iloc[y_step]+0.005, y_step, 
                                     '**',
                                     size= size, rotation=0.,
                                     ha="left", va="center", color = 'black',

                                     )
            elif(df.sort_values('Estimate', ascending=False)['p value'].iloc[y_step]>= 0.01 and df.sort_values('Estimate', ascending=False)['p value'].iloc[y_step]<0.05):
                  
                    plt.text(df.sort_values('Estimate', ascending=False)['Estimate'].iloc[y_step]+0.005, y_step, 
                                     '*',
                                     size= size, rotation=0.,
                                     ha="left", va="center", color = 'black',

                                     )  
            else:
                    plt.text(df.sort_values('Estimate', ascending=False)['Estimate'].iloc[y_step]+0.005, y_step, 
                                     ' ',
                                     size= size, rotation=0.,
                                     ha="left", va="center", color = 'black',

                                     )    

            y_step+=1
            
         
    text_diff =xlim[1]
    #coef_SVR_tot.loc['R-squared', 'value']
    f_prob = '(ns)'
    if(regression_model.loc['F-pvalue', 'value']<0.01):
        f_prob = '(**)'
    elif(regression_model.loc['F-pvalue', 'value']<0.05):
        f_prob = '(*)'
        
    
    plot_ci = np.round(regression_model.loc['CI', 'value'],2)
    
    plt.text(xlim[1] - 0.05*xlim[1], df.shape[0]- 0.25*df.shape[0],
             r'$ R^2 = $' + str(np.round(regression_model.loc['R-squared', 'value'],2))+ '\n$CI = $' + str(plot_ci) + '\n$F^2 = $' + str(np.round(regression_model.loc['F-squared', 'value'],2)) + ' ' + '\n$F = $' + str(np.round(regression_model.loc['F', 'value'],2)) + '\n$P (F) = $' + f_prob,
                             size= 12, rotation=0.,
                             ha="right", va="center", color = 'black',
                             bbox=dict(boxstyle="round",
                                       ec='gray',
                                        fc='gray',
                                       )
                             )

    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)
    
    plt.xlabel('Estimate', fontsize=12)
    plt.ylabel('Features', fontsize=12)
    plt.locator_params(axis='x', nbins=4)
    

def plot_evolution(dict_Df, i, X_cat_, xlim = [0,25], size = 10):

    fp_value = dict_Df[i][1]
    fp_value_fdr = dict_Df[i][2]
    coef_df_ = dict_Df[i][0]


    title = 'iter:' + str(i) + '\n'
    title += '|    f-pva: (' + str(np.min(fp_value).round(3)) + ' ; ' + str(np.mean(fp_value).round(3)) + ' ; ' + str(np.max(fp_value).round(3)) + ' ; ' + str(np.median(fp_value).round(3)) + ') |\n'
    title += '|  fdr-pva: (' + str(np.min(fp_value_fdr).round(3)) + ' ; ' + str(np.mean(fp_value_fdr).round(3)) + ' ; ' + str(np.max(fp_value_fdr).round(3)) + ' ; ' + str(np.median(fp_value_fdr).round(3)) + ') |'



    coef_df_ = coef_df_.sort_values('Estimate', ascending=False)
    
    bar_color = get_bar_colors(coef_df_, X_cat_)
    plt.title(title)
    sns.barplot(x="Estimate", y="Features", data = coef_df_, xerr=coef_df_['Estimate std'], palette =bar_color)
    plt.xlim(xlim);

    y_step=0 
    for j in range(coef_df_.shape[0]):
        if(coef_df_['p value'].iloc[y_step]<0.01):

            plt.text(coef_df_['Estimate'].iloc[y_step]+0.005, y_step, 
                                             '**',
                                             size= size, rotation=0.,
                                             ha="left", va="center", color = 'black',

                                             )
        elif(coef_df_['p value'].iloc[y_step]>= 0.01 and df.sort_values('Estimate', ascending=False)['p value'].iloc[y_step]<0.05):

            plt.text(coef_df_['Estimate'].iloc[y_step]+0.005, y_step, 
                                             '*',
                                             size= size, rotation=0.,
                                             ha="left", va="center", color = 'black',

                                             )  
        else:
            plt.text(coef_df_['Estimate'].iloc[y_step]+0.005, y_step, 
                                             ' ',
                                             size= size, rotation=0.,
                                             ha="left", va="center", color = 'black',

                                             )    

        y_step+=1
        
        
    
def SearchBestModel(case_x, case_y, case_x_for_strat):
    X_train, X_test, y_train, y_test = train_test_split(case_x, case_y, test_size=.30, stratify=case_x_for_strat, random_state=1)

    opt_SVR = BayesSearchCV(
            SVR(),
            {
                #'gamma': ( 0.0001, 0.01, 0.001, 0.1, 1, 10),
                'kernel': ['rbf'],
                'degree': (3, 4, 5, 6),
                #'tol:' : (1e-6, 1e-3, 1e+1),
                #'n_estimators': (100, 1000),

            },
            n_iter=10,
            cv=3, random_state=1
        )

    opt_SVR.fit(X_train, y_train)


    print('SVR')
    print("val. score: %s" % opt_SVR.best_score_)
    print("test score: %s" % opt_SVR.score(X_test, y_test))
    print("best parameters: %s" % str(opt_SVR.best_params_))
    print('---------------------------------------------\n')
    
    return opt_SVR

def all_process(conn_z_score_fer, conn_z_score_tom, conn_z_score_tot, iter_tot = 1000, model_name = '', best_ = '', xl = 80):

    ## y data
    y_2A_fer = conn_z_score_fer['mini_sea_fer']
    y_2A_tom = conn_z_score_tom['mini_sea_tom']
    y_2A_tot = conn_z_score_tot['mini_sea_total']



    ## x data
    X_2A_fer = conn_z_score_fer.drop(['mini_sea_fer'], axis =1)
    X_2A_tom = conn_z_score_tom.drop(['mini_sea_tom'], axis =1)
    X_2A_tot = conn_z_score_tot.drop(['mini_sea_total'], axis =1)


    best_w_cog = list(best_)
    #best_w_cog.extend(['Edad_procesada', 'years_education', 'diagnosis_specify', 'country_class','cognition_general_score', 'executive_score'])#, 'gender'])
    best_w_cog = np.array(best_w_cog)
    
    #print(best_w_cog)


    X_2A_fer = X_2A_fer[best_w_cog]
    X_2A_tom = X_2A_tom[best_w_cog]
    X_2A_tot = X_2A_tot[best_w_cog]
    
    case2A_fer_for_strat =  y_2A_fer.copy()
    median = case2A_fer_for_strat.median()
    case2A_fer_for_strat.loc[case2A_fer_for_strat<median]= 0
    case2A_fer_for_strat.loc[case2A_fer_for_strat>=median]=1 
    
    case2A_tom_for_strat =  y_2A_tom.copy()
    median = case2A_tom_for_strat.median()
    case2A_tom_for_strat.loc[case2A_tom_for_strat<median]= 0
    case2A_tom_for_strat.loc[case2A_tom_for_strat>=median]=1 

    case2A_tot_for_strat =  y_2A_tot.copy()
    median = case2A_tot_for_strat.median()
    case2A_tot_for_strat.loc[case2A_tot_for_strat<median]= 0
    case2A_tot_for_strat.loc[case2A_tot_for_strat>=median]=1 
    
    # Boostsrapping fer
    
    X_2A_fer_classic_reg = X_2A_fer

    run = True

    coef_SVR_fer_dir_2A = {}


    run_count = 0
    while run:
        coef_SVR_fer = boostrapping(X_2A_fer_classic_reg, y_2A_fer, case2A_fer_for_strat, list(X_2A_fer_classic_reg.columns), iter_total=iter_tot)

        coef_SVR_fer_dir_2A[run_count] = coef_SVR_fer

        run_count+=1

        df = coef_SVR_fer[0]
        df.index.name = 'Features'
        df = df.iloc[1:-5, 0:-1]

        df = df.reset_index()
        data = df.sort_values('p value', ascending=False)
        data

        feat_remove = data.iloc[0,0]

        if(X_2A_fer_classic_reg.shape[1]<=8 or data.iloc[0,3] < 0.05 or coef_SVR_fer[0].loc['F-pvalue', 'value'] < 0.05):
            run = False
        else:
            X_2A_fer_classic_reg = X_2A_fer_classic_reg.drop([feat_remove], axis=1)
            print(feat_remove)
            
            
    # Boostsrapping tom
        
    X_2A_tom_classic_reg = X_2A_tom

    run = True

    coef_SVR_tom_dir_2A = {}


    run_count = 0
    while run:
        coef_SVR_tom = boostrapping(X_2A_tom_classic_reg, y_2A_tom, case2A_tom_for_strat, list(X_2A_tom_classic_reg.columns), iter_total=iter_tot)

        coef_SVR_tom_dir_2A[run_count] = coef_SVR_tom

        run_count+=1

        df = coef_SVR_tom[0]
        df.index.name = 'Features'
        df = df.iloc[1:-5, 0:-1]

        df = df.reset_index()
        data = df.sort_values('p value', ascending=False)
        data

        feat_remove = data.iloc[0,0]

        if(X_2A_tom_classic_reg.shape[1]<=8 or data.iloc[0,3] < 0.05 or coef_SVR_tom[0].loc['F-pvalue', 'value'] < 0.05):
            run = False
        else:
            X_2A_tom_classic_reg = X_2A_tom_classic_reg.drop([feat_remove], axis=1)
            print(feat_remove)
            
     # Boostsrapping tot
    X_2A_tot_classic_reg = X_2A_tot

    run = True

    coef_SVR_tot_dir_2A = {}


    run_count = 0
    while run:
        coef_SVR_tot = boostrapping(X_2A_tot_classic_reg, y_2A_tot, case2A_tot_for_strat, list(X_2A_tot_classic_reg.columns), iter_total=iter_tot)

        coef_SVR_tot_dir_2A[run_count] = coef_SVR_tot

        run_count+=1

        df = coef_SVR_tot[0]
        df.index.name = 'Features'
        df = df.iloc[1:-5, 0:-1]

        df = df.reset_index()
        data = df.sort_values('p value', ascending=False)
        data

        feat_remove = data.iloc[0,0]

        if(X_2A_tot_classic_reg.shape[1]<=8 or data.iloc[0,3] < 0.05 or data.iloc[0,3] < 0.05 or coef_SVR_tot[0].loc['F-pvalue', 'value'] < 0.01):
            run = False
        else:
            X_2A_tot_classic_reg = X_2A_tot_classic_reg.drop([feat_remove], axis=1)
            print(feat_remove)
            
    fer_2A_keys = list(coef_SVR_fer_dir_2A.keys())
    tom_2A_keys = list(coef_SVR_tom_dir_2A.keys())
    tot_2A_keys = list(coef_SVR_tot_dir_2A.keys())

    

    plt.figure(figsize=(13,10))
    grid = plt.GridSpec(2, 3)

    plt.subplot(grid[0, 0])
    plot_estimate_value(coef_SVR_fer_dir_2A[fer_2A_keys[-1]][0], X_cat, title = 'SVR fer (' + str(X_2A_fer_classic_reg.shape[0]) + ')',  xlim =[0, xl])
    plt.subplot(grid[0, 1])
    plot_estimate_value(coef_SVR_tom_dir_2A[tom_2A_keys[-1]][0], X_cat, title = 'SVR tom (' + str(X_2A_tom_classic_reg.shape[0]) + ')',  xlim =[0, xl])
    plt.subplot(grid[0, 2])
    plot_estimate_value(coef_SVR_tot_dir_2A[tot_2A_keys[-1]][0], X_cat, title = 'SVR tot (' + str(X_2A_tot_classic_reg.shape[0]) + ')',  xlim =[0, xl])

    boxplots = []
    boxplots.append(coef_SVR_fer_dir_2A[fer_2A_keys[-1]][1])
    boxplots.append(coef_SVR_tom_dir_2A[tom_2A_keys[-1]][1])
    boxplots.append(coef_SVR_tot_dir_2A[tot_2A_keys[-1]][1])



    plt.subplot(grid[1, :])
    plt.boxplot(boxplots);
    plt.xticks(list(range(1,4)), ['fer', 'tom', 'tot'], rotation = 45);
    plt.ylabel('R2')
    plt.xlabel('Outcomes')

    plt.tight_layout(pad=2);
    
    #plt.savefig('figures/' + model_name +'.pdf', dpi=300, bbox_inches='tight')
    
     
    coef_SVR_fer_dir_2A[fer_2A_keys[-1]][0].to_excel('results/'+model_name+'_fer.xlsx')
    coef_SVR_tom_dir_2A[tom_2A_keys[-1]][0].to_excel('results/'+model_name+'_tom.xlsx')
    coef_SVR_tot_dir_2A[tot_2A_keys[-1]][0].to_excel('results/'+model_name+'_tot.xlsx')


### Datos

In [ ]:
conn_z_score = pd.read_excel('../Data/Base.xlsx', index_col=0)
X_cat = pd.read_csv('../Data/var_name_color_code.csv', encoding='latin-1', sep=";")

In [ ]:
#colunm = 'diagnosis_specify'
#cn_col = conn_z_score[colunm]

#conn_z_score[colunm] = conn_z_score[colunm].apply(lambda x: (x-cn_col.mean())/ cn_col.std())


colunm = 'country_class'
cn_col = conn_z_score[colunm]

conn_z_score[colunm] = conn_z_score[colunm].apply(lambda x: (x-cn_col.mean())/ cn_col.std())



colunm = 'Edad_procesada'
cn_col = conn_z_score[colunm]

conn_z_score[colunm] = conn_z_score[colunm].apply(lambda x: (x-cn_col.mean())/ cn_col.std())



colunm = 'years_education'
cn_col = conn_z_score[colunm]

conn_z_score[colunm] = conn_z_score[colunm].apply(lambda x: (x-cn_col.mean())/ cn_col.std())


colunm = 'cognition_general_score'
cn_col = conn_z_score[colunm]

conn_z_score[colunm] = conn_z_score[colunm].apply(lambda x: (x-cn_col.mean())/ cn_col.std())



colunm = 'executive_score'
cn_col = conn_z_score[colunm]

conn_z_score[colunm] = conn_z_score[colunm].apply(lambda x: (x-cn_col.mean())/ cn_col.std())

In [ ]:
conn_z_score = conn_z_score.copy()

In [ ]:
conn_z_score.reset_index(inplace=True, drop=True)

In [ ]:
conn_z_score.columns

## Bulletpoint 1

In [ ]:
best = ['gender', 'Edad_procesada',
       'years_education', 'cognition_general_score', 'executive_score']

In [ ]:
best

In [ ]:
conn_z_score_qcs_dcl = conn_z_score[(conn_z_score['diagnosis_specify']==4 ) | (conn_z_score['diagnosis_specify']==5.000000000000001)]
conn_z_score_qcs_dcl.reset_index(inplace=True, drop=True)

In [ ]:
sex_qcs_dcl_fer = conn_z_score_qcs_dcl.drop([              'mini_sea_tom', 'mini_sea_total'], axis =1 )
sex_qcs_dcl_tom = conn_z_score_qcs_dcl.drop(['mini_sea_fer',                 'mini_sea_total'], axis =1 )
sex_qcs_dcl_tot = conn_z_score_qcs_dcl.drop(['mini_sea_fer', 'mini_sea_tom',                 ], axis =1 )

sex_qcs_dcl_fer.dropna(inplace=True)
sex_qcs_dcl_tom.dropna(inplace=True)  
sex_qcs_dcl_tot.dropna(inplace=True) 

print(sex_qcs_dcl_fer.shape,
sex_qcs_dcl_tom.shape,  
sex_qcs_dcl_tot.shape)

In [ ]:
all_process(sex_qcs_dcl_fer, sex_qcs_dcl_tom, sex_qcs_dcl_tot, iter_tot = 100, model_name = '', best_=best, xl = 20)